In [1]:
import math
import robodk
import robolink
import numpy as np

In [2]:
RDK = robolink.Robolink()
robot = RDK.Item('UR5e', robolink.ITEM_TYPE_ROBOT)
robot.setSpeed(100)
home = RDK.Item('home')
imu_frame = RDK.Item('cf_imu_frame', robolink.ITEM_TYPE_FRAME)
camera_frame = RDK.Item('cf_camera_frame', robolink.ITEM_TYPE_FRAME)
charuco_frame = RDK.Item('charuco_frame', robolink.ITEM_TYPE_FRAME)
tcp_frame = RDK.Item('tcp_frame', robolink.ITEM_TYPE_FRAME)
base_frame = RDK.Item('UR5e Base', robolink.ITEM_TYPE_FRAME)

In [3]:

home_pose = home.Pose()
circle_center = home_pose.Pos()

circle_center

[-136.0000000000004, -430.00000000000057, 50.00000000000378]

In [4]:
    
    # 圆的参数
    R = 150  # radius
    num_points = 8  # number of points
    height = circle_center[2] + 220  # 比基准点高220mm

    delta_x = 20
targets = []
for i in range(R//delta_x):
    radius = math.sqrt((R**2 - (delta_x*i)**2))
    if i != 0:
        height += delta_x
        num_points -= 1


    for j in range(num_points):
        angle = 2 * np.pi * j / num_points
        x = circle_center[0] + radius * np.cos(angle)
        y = circle_center[1] + radius * np.sin(angle)
        z = height
        targets.append([x, y, z])
targets

[[13.999999999999602, -430.00000000000057, 270.00000000000375],
 [-29.933982822018265, -323.93398282201844, 270.00000000000375],
 [-136.0000000000004, -280.00000000000057, 270.00000000000375],
 [-242.06601717798253, -323.93398282201844, 270.00000000000375],
 [-286.0000000000004, -430.00000000000057, 270.00000000000375],
 [-242.06601717798253, -536.0660171779826, 270.00000000000375],
 [-136.00000000000043, -580.0000000000006, 270.00000000000375],
 [-29.933982822018294, -536.0660171779828, 270.00000000000375],
 [12.660687473184652, -430.00000000000057, 290.00000000000375],
 [-43.311577423161125, -313.7723943281238, 290.00000000000375],
 [-169.0801150191213, -285.0665463382537, 290.00000000000375],
 [-269.9386512943113, -365.4985450593483, 290.00000000000375],
 [-269.9386512943113, -494.5014549406528, 290.00000000000375],
 [-169.08011501912134, -574.9334536617474, 290.00000000000375],
 [-43.31157742316117, -546.2276056718773, 290.00000000000375],
 [8.568322948009211, -430.00000000000057, 

In [42]:
math.sqrt(radius**2 - delta_x**2)

140.35668847618197

In [44]:
for i, target_pos in enumerate(targets):
    target_name = f"Target_{i+1}"
    target_item = RDK.AddTarget(target_name)
    target_item.setParent(base_frame)
    # 设置目标点的位置
    rx, ry, rz = np.radians(180), np.radians(0), np.radians(0)  # 旋转角度，转换为弧度
    target_pose = robodk.Mat([[1, 0, 0, target_pos[0]],
                           [0, 1, 0, target_pos[1]],
                           [0, 0, 1, target_pos[2]],
                           [0, 0, 0, 1]])

    # 可以使用robodk的函数来生成基于rx, ry, rz的旋转矩阵
    rotm = robodk.rotz(rz) * robodk.roty(ry) * robodk.rotx(rx)

    target_pose[:3,:3] = rotm[:3, :3]

    # 设置目标的姿态
    target_item.setPose(target_pose)


In [5]:
RDK = robolink.Robolink()
robot = RDK.Item('UR5e', robolink.ITEM_TYPE_ROBOT)
robot.setSpeed(100)
for i in range(len(targets)):
    target = (RDK.Item(f'Target_{i + 1}'))
    robot.MoveL(target)

StoppedError: Robot stopped by user

In [7]:
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
       return v
    return v / norm

def calculate_target_vector(frame, target):
    # 计算从frame指向target的方向向量
    return normalize(np.array(target) - np.array(frame[:3]))

def align_z_axis_to_vector(vector):
    """
    Create a rotation matrix that aligns the Z-axis with the given vector.
    """
    z_axis = vector
    x_axis = normalize(np.cross([0, 1, 0], z_axis))
    y_axis = np.cross(z_axis, x_axis)

    # 构造旋转矩阵
    R = np.array([x_axis, y_axis, z_axis]).T
    return R

def rotation_matrix_to_xyz_euler(R):
    """
    Convert a rotation matrix to XYZ Euler angles in degrees.
    """
    # 假设旋转顺序为X-Y-Z
    if R[2,0] < 1:
        if R[2,0] > -1:
            y = np.arcsin(-R[2,0])
            x = np.arctan2(R[2,1]/np.cos(y), R[2,2]/np.cos(y))
            z = np.arctan2(R[1,0]/np.cos(y), R[0,0]/np.cos(y))
        else: # R[2,0] == -1
            y = np.pi/2
            x = -np.arctan2(-R[0,1], R[0,2])
            z = 0
    else: # R[2,0] == 1
        y = -np.pi/2
        x = np.arctan2(-R[0,1], R[0,2])
        z = 0

    return np.degrees([x, y, z])

def calculate_orientation_to_target(frame, target):
    direction_vector = calculate_target_vector(frame, target)
    R = align_z_axis_to_vector(direction_vector)
    euler_angles = rotation_matrix_to_xyz_euler(R)
    return euler_angles

for i in range(len(targets)):
    frame = targets[i]
    target = [-136, -430, 0] # Target frame position and orientation (degrees)
     # Frame position and orientation (degrees)

    # Calculate the orientation
    orientation = calculate_orientation_to_target(frame, target)
    print("Orientation (Euler angles):", orientation)
    RDK = robolink.Robolink()
    robot = RDK.Item('UR5e', robolink.ITEM_TYPE_ROBOT)
    base_frame = RDK.Item('UR5e Base', robolink.ITEM_TYPE_FRAME)

    target = RDK.AddTarget(f'r_{int(R)} delta_x_{delta_x} _{i+1}')
    target.setParent(base_frame)

    # 定义位置和姿态
    x, y, z = frame[0], frame[1], frame[2]  # 单位通常是毫米
    rx, ry, rz = np.radians(orientation[0]), np.radians(orientation[1]), np.radians(orientation[2])  # 旋转角度，转换为弧度
    target_pose = robodk.Mat([[1, 0, 0, x],
                           [0, 1, 0, y],
                           [0, 0, 1, z],
                           [0, 0, 0, 1]])

    # 可以使用robodk的函数来生成基于rx, ry, rz的旋转矩阵
    rotm = robodk.rotz(rz) * robodk.roty(ry) * robodk.rotx(rx)

    print("rotm", rotm)
    # 将旋转矩阵应用到目标姿态
    target_pose[:3,:3] = rotm[:3, :3]

    # 设置目标的姿态
    target.setPose(target_pose)

Orientation (Euler angles): [180.        -29.0546041 180.       ]
rotm Pose(0.000, 0.000, 0.000,  -180.000, -29.055, -180.000):
[[ -0.874, 0.000, -0.486, 0.000 ],
 [ 0.000, 1.000, 0.000, 0.000 ],
 [ 0.486, 0.000, -0.874, 0.000 ],
 [ 0.000, 0.000, 0.000, 1.000 ]]

Orientation (Euler angles): [-159.91575761  -21.44674152  180.        ]
rotm Pose(0.000, 0.000, 0.000,  158.553, -20.084, 172.317):
[[ -0.931, -0.126, -0.343, 0.000 ],
 [ 0.000, 0.939, -0.343, 0.000 ],
 [ 0.366, -0.320, -0.874, 0.000 ],
 [ 0.000, 0.000, 0.000, 1.000 ]]

Orientation (Euler angles): [-1.50945396e+02  8.44378369e-14  1.80000000e+02]
rotm Pose(0.000, 0.000, 0.000,  150.945, 0.000, -180.000):
[[ -1.000, 0.000, 0.000, 0.000 ],
 [ 0.000, 0.874, -0.486, 0.000 ],
 [ -0.000, -0.486, -0.874, 0.000 ],
 [ 0.000, 0.000, 0.000, 1.000 ]]

Orientation (Euler angles): [-159.91575761   21.44674152  180.        ]
rotm Pose(0.000, 0.000, 0.000,  158.553, 20.084, -172.317):
[[ -0.931, 0.126, 0.343, 0.000 ],
 [ 0.000, 0.939, -0.343,